In [1]:
import at
import sys
from importlib.resources import files, as_file
import numpy as np

In [2]:
fname = "hmba.mat"
with as_file(files("machine_data") / fname) as path:
    ring = at.load_lattice(path)

In [3]:
from at.future import VariableBase, ElementVariable, RefptsVariable, CustomVariable

# Variables

Variables are **references** to any scalar quantity. Predefined classes are available
for accessing any scalar attribute of an element, or any item of an array attribute.

Any other quantity may be accessed by either subclassing the {py:class}`~.variables.VariableBase`
abstract base class, or using a {py:class}`~.variables.CustomVariable`.

## {py:class}`~.lattice_variables.ElementVariable`

An {py:class}`~.lattice_variables.ElementVariable` refers to a single attribute (or item of an array attribute) of one or several {py:class}`.Element` objects.

We now create a variable pointing to the length of all QF1 magnets of *ring*:

In [4]:
lf1 = ElementVariable(ring["QF1[AE]"], "Length", name="lf1")
print(f"lf1: {lf1}")
print(lf1.value)

lf1: ElementVariable(0.311896, name='lf1')
0.311896


and another variable pointing to the strength of the same magnets:

In [5]:
kf1 = ElementVariable(ring["QF1[AE]"], "PolynomB", index=1, name="kf1")
print("kf1:", kf1)
print(kf1.value)

kf1: ElementVariable(2.5394599781303304, name='kf1')
2.5394599781303304


We can check which elements are concerned by the `kf1` variable. The element container is a set, so that no element may appear twice:

In [6]:
kf1.elements

{Quadrupole('QF1A', 0.311896, 2.5394599781303304, FringeQuadEntrance=1, FringeQuadExit=1, NumIntSteps=20),
 Quadrupole('QF1E', 0.311896, 2.5394599781303304, FringeQuadEntrance=1, FringeQuadExit=1, NumIntSteps=20)}

`kf1` drives 2 quadrupoles. Let's look at the 1{sup}`st` one:

In [7]:
print(ring[5])

Quadrupole:
	FamName : QF1A
	Length : 0.311896
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 20
	FringeQuadEntrance : 1
	FringeQuadExit : 1
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : [0.         2.53945998]
	K : 2.5394599781303304


We can now change the strength of both QF1 magnets and check again the 1{sup}`st` one:

In [8]:
kf1.set(2.5)
print(ring[5])

Quadrupole:
	FamName : QF1A
	Length : 0.311896
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 20
	FringeQuadEntrance : 1
	FringeQuadExit : 1
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : [0.  2.5]
	K : 2.5


We can look at the history of `kf1` values

In [9]:
kf1.history

[2.5394599781303304, 2.5]

And revert to the initial or previous values:

In [10]:
kf1.set_previous()
print(ring[5])

Quadrupole:
	FamName : QF1A
	Length : 0.311896
	PassMethod : StrMPoleSymplectic4Pass
	NumIntSteps : 20
	FringeQuadEntrance : 1
	FringeQuadExit : 1
	MaxOrder : 1
	PolynomA : [0. 0.]
	PolynomB : [0.         2.53945998]
	K : 2.5394599781303304


An {py:class}`~.lattice_variables.ElementVariable` is linked to Elements. It will not follow any copy of the element, neither shallow nor deep. So if we make a copy of ring:

In [11]:
newring = ring.deepcopy()
print(f"ring:   {ring[5].PolynomB[1]}")
print(f"newring: {newring[5].PolynomB[1]}")

ring:   2.5394599781303304
newring: 2.5394599781303304


and modify the `kf1` variable:

In [12]:
kf1.set(2.6)

In [13]:
print(f"ring:   {ring[5].PolynomB[1]}")
print(f"newring: {newring[5].PolynomB[1]}")

ring:   2.6
newring: 2.5394599781303304


The QF1 in `newring` is not affected.

One can set upper and lower bounds on a variable. Trying to set a value out of the bounds will raise a {py:obj}`ValueError`. The default is (-{py:obj}`numpy.inf`, {py:obj}`numpy.inf`).

In [14]:
lfbound = ElementVariable(ring["QF1[AE]"], "Length", bounds=(0.30, 0.35))

In [15]:
lfbound.set(0.2)

ValueError: set value must be in (0.3, 0.35)

Variables also accept a *delta* keyword argument. Its value is used as the initial step in matching, and in the {py:meth}`~.variables.VariableBase.step_up` and {py:meth}`~.variables.VariableBase.step_down` methods.

## {py:class}`.RefptsVariable`

An {py:class}`.RefptsVariable` is similar to an {py:class}`~.lattice_variables.ElementVariable` but it is not associated with an {py:class}`~.Element`
itself, but with its location in a Lattice. So it will act on any lattice with the same elements.

But it needs a *ring* keyword in its *set* and *get* methods, to identify the selected lattice.

In [16]:
kf2 = RefptsVariable("QF1[AE]", "PolynomB", index=1, name="kf2")

We can now use this variable on the two rings:

In [17]:
kf2.set(2.55, ring=ring)
kf2.set(2.45, ring=newring)

In [18]:
print(f"ring:   {ring[5].PolynomB[1]}")
print(f"newring: {newring[5].PolynomB[1]}")

ring:   2.55
newring: 2.45


## Custom variables
We take the example of a variable driving the length of two drifts (or other elements), such
that their sum stays constant. We choose the variable value as the length of the first element, the other one will be automatically adjusted. This variable allows to shift longitudinally the intermediate
part of the lattice without affecting its circumference, so we will call the variable an "element shifter".

Note that defining correlated {py:class}`.Element` attributes may be easier done with Parameters. However Variables are not restricted to Element attributes, unlike Parameters.

Similarly to the {py:class}`~.lattice_variables.RefptsVariable`, we will refer to the 2 variable elements by their `refpts`. Alternatively, one could give the elements themselves, as in {py:class}`~.lattice_variables.ElementVariable`.

### Using the {py:class}`~.variables.CustomVariable`

We need to define two functions for the "get" end "set" actions, and to give to the {py:class}`~.variables.CustomVariable` constructor the necessary arguments for these functions.

We start with the "set" function. Since the elements may appear several times in the
lattice, we set them all. We are free to declare any need argument.

In [19]:
def setvar(value, ref1, ref2, total_length, ring=None):
    if ring is None:
        raise ValueError("Can't set values if ring is None")
    for elem in ring.select(ref1):
        elem.Length = value
    for elem in ring.select(ref2):
        elem.Length = total_length - value

In the "get" function, since we chose the first element as the variable value, we may ignore
the other arguments:

In [20]:
def getvar(ref1, ref2, total_length, ring=None):
    if ring is None:
        raise ValueError("Can't get values if ring is None")
    return np.mean([elem.Length for elem in ring.select(ref1)])

We can now select the elements, get the initial conditions and construct the variable:

In [21]:
# select the variable elements
elem1 = "DR_01"
elem2 = "DR_02"
# Compute the initial total length
l1 = np.mean([elem.Length for elem in ring.select(elem1)])
l2 = np.mean([elem.Length for elem in ring.select(elem2)])
# Create the variable
elem_shifter1 = CustomVariable(
    setvar, getvar, elem1, elem2, l1 + l2, bounds=(0, l1 + l2)
)

Here is the initial state of the lattice:

In [22]:
print(f"{ring[2]}\n{ring[4]}")
print("\nelem_shifter1.get:", elem_shifter1.get(ring=ring, initial=True))

Drift:
	FamName : DR_01
	Length : 2.6513999999999998
	PassMethod : DriftPass
Drift:
	FamName : DR_02
	Length : 0.042552
	PassMethod : DriftPass

elem_shifter1.get: 2.6513999999999998


Now, let's set a new value for the variable and look at the result:

In [23]:
elem_shifter1.set(2.5, ring=ring)
print(f"{ring[2]}\n{ring[4]}")
print("\nelem_shifter1.get:", elem_shifter1.get(ring=ring))

Drift:
	FamName : DR_01
	Length : 2.5
	PassMethod : DriftPass
Drift:
	FamName : DR_02
	Length : 0.1939519999999999
	PassMethod : DriftPass

elem_shifter1.get: 2.5


We can look at the history of the variable

In [24]:
elem_shifter1.history

[2.6513999999999998, 2.5]

and go back to the initial value

In [25]:
elem_shifter1.reset(ring=ring)

### By derivation of the {py:class}`~.variables.VariableBase` class

We will write a new variable class based on {py:class}`~.variables.VariableBase` abstract base class. The main task is to implement the `_setfun` and `_getfun` abstract methods.

In [26]:
class ElementShifter(VariableBase):
    def __init__(self, ref1, ref2, total_length=None, **kwargs):
        """Varies the length of the elements *dr1* and *dr2*
        keeping the sum of their lengths equal to *total_length*.

        If *total_length* is None, it is set to the initial total length
        """
        # Store the indices of the 2 variable elements
        self.ref1 = ref1
        self.ref2 = ref2
        # Compute and store the initial total length
        if total_length is None:
            l1 = np.mean([elem.Length for elem in ring.select(ref1)])
            l2 = np.mean([elem.Length for elem in ring.select(ref2)])
            self.total_length = l1 + l2
        self.length = total_length
        # Initialise the parent class
        super().__init__(bounds=(0.0, self.total_length), **kwargs)

    def _setfun(self, value, ring=None):
        if ring is None:
            raise ValueError("Can't get values if ring is None")
        for elem in ring.select(self.ref1):
            elem.Length = value
        for elem in ring.select(self.ref2):
            elem.Length = self.total_length - value

    def _getfun(self, ring=None):
        if ring is None:
            raise ValueError("Can't get values if ring is None")
        return np.mean([elem.Length for elem in ring.select(self.ref1)])

We construct the variable:

In [27]:
elem_shifter2 = ElementShifter(elem1, elem2)

Look at the initial state:

In [28]:
print(f"{ring[2]}\n{ring[4]}")
print("\nelem_shifter2.get:", elem_shifter2.get(ring=ring, initial=True))

Drift:
	FamName : DR_01
	Length : 2.6513999999999998
	PassMethod : DriftPass
Drift:
	FamName : DR_02
	Length : 0.042552000000000145
	PassMethod : DriftPass

elem_shifter2.get: 2.6513999999999998


Change the variable, and look at the result:

In [29]:
elem_shifter2.set(2.5, ring=ring)
print(f"{ring[2]}\n{ring[4]}")
print("\nelem_shifter2.get:", elem_shifter2.get(ring=ring))

Drift:
	FamName : DR_01
	Length : 2.5
	PassMethod : DriftPass
Drift:
	FamName : DR_02
	Length : 0.1939519999999999
	PassMethod : DriftPass

elem_shifter2.get: 2.5


Both variables behave similarly. But the derivation allows more control by making use of the
`__init__` method. For instance here it includes the computation of the initial total length.